## Import Packages

In [ ]:
# ---- Load Libraries ----
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from matplotlib import colors
from scipy.integrate import trapezoid
import pickle
import sklearn
import gc

In [ ]:
# block cache for faster pings to s3
import fsspec
fsspec.config.conf["s3"] = {
    "default_cache_type": "blockcache",
    # "default_cache_size": 2**22,
}

## Query Data

In [ ]:
# eartdata login

import earthaccess
auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

In [ ]:
# 8 day average
results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    temporal = ("2024-08-15", "2025-01-15"),
    granule_name="*.8D.*.4km.*"
)

In [ ]:
fileset = earthaccess.open(results)

In [ ]:
len(fileset)

## Open Data

In [ ]:
bbox = (-83.5, 25.0, -81.5, 30.13721061020233)

In [ ]:
# to add time/day axis
def time_from_attr(ds):
    """Set the time attribute as a dataset variable
    Args:
        ds: a dataset corresponding to one or multiple Level-2 granules
    Returns:
        the dataset with a scalar "time" coordinate
    """
    datetime = ds.attrs["time_coverage_start"].replace("Z", "")
    ds["date"] = ((), np.datetime64(datetime, "ns"))
    ds = ds.set_coords("date")
    return ds

In [ ]:
# Load dataset
dataset = xr.open_mfdataset(fileset, combine="nested", concat_dim="date", preprocess=time_from_attr)

In [ ]:
# Assign core variables
Rrs = dataset["Rrs"].sel({"lon": slice(bbox[0],bbox[2]), 
                          "lat": slice(bbox[3],bbox[1]),
                         })
latitude = Rrs["lat"]
longitude = Rrs["lon"]
wavelengths = dataset["wavelength"]
date = dataset["date"]

In [ ]:
%%time
# ---- Make product plot ----
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=cartopy.crs.PlateCarree())
ax.gridlines(draw_labels={"bottom": "x", "left": "y"})
ax.coastlines()

# ---- Adjust colorbar limits ----
# ndci = (Rrs[0].sel(wavelength=709, method="nearest") - Rrs[0].sel(wavelength=665, method="nearest"))/(Rrs[0].sel(wavelength=709, method="nearest") + Rrs[0].sel(wavelength=665, method="nearest"))
rbd = (Rrs[0].sel(wavelength=678, method="nearest") - Rrs[0].sel(wavelength=667, method="nearest"))
kbbi = (Rrs[0].sel(wavelength=678, method="nearest") - Rrs[0].sel(wavelength=667, method="nearest"))/(Rrs[0].sel(wavelength=678, method="nearest") + Rrs[0].sel(wavelength=667, method="nearest"))


# test = xr.DataArray(
#     rrs[0][:,:,0],
#     coords={"lat": latitude, "lon": longitude},
#     dims=["lat", "lon"]
# )

plot = rbd.plot(x="lon", y="lat", cmap="jet",)

# ---- Customize colorbar label ----
plot.colorbar.set_label("KBBI", labelpad=10, fontsize=16, fontweight="bold")
#plt.title('')

## Prepare Data for Model Inference

In [ ]:
feature_list = [585, 588, 590, 593, 595, 598, 600, 602, 605, 607, 610, 612, 615, 617, 620, 622, 624, 627, 629, 632, 634, 637, 639, 641, 642, 643, 644, 645, 647, 648, 649, 650, 652, 653, 654, 655, 657, 658, 659, 660, 662, 663, 664, 665, 667, 668, 669, 670, 671, 673, 674, 675, 676, 678, 679, 680, 681, 683, 684, 685, 686, 688, 689, 690, 691, 693, 694, 695, 696, 698, 699, 700, 701, 703, 704, 705, 706, 708, 709, 710, 711, 713, 714, 716, 719, 998, 999]

In [ ]:
# create additional features as k.brevis indices
rbd = Rrs.sel(wavelength=678, method="nearest") - Rrs.sel(wavelength=667, method="nearest")
ss_490 = Rrs.sel(wavelength=487, method="nearest") - Rrs.sel(wavelength=444, method="nearest") - (Rrs.sel(wavelength=532, method="nearest") - Rrs.sel(wavelength=444, method="nearest"))*(487-444)/(532-444)

# add those features as bands in the Rrs stack 
rbd = rbd.expand_dims(dim={"wavelength": [998]})
ss_490 = ss_490.expand_dims(dim={"wavelength": [999]})
Rrs = xr.concat([Rrs, rbd, ss_490], dim="wavelength")

In [ ]:
# select only model specific bands and features
Rrs = Rrs.sel(wavelength=feature_list, method="nearest")

## Inferring K.brevis Cell count from OCI Weekly Hyperspectral Rrs Averages

In [ ]:
%%time

# saving all values in memory for faster inferencing and avoiding callbacks
rrs = Rrs.values
rrs.shape

In [ ]:
# saving np array for future inferences
np.savez("8D_rrs_Aug152024-Jan152025.npz", rrs=rrs, latitude=latitude, longitude=longitude)

In [ ]:
# loading np array
data = np.load('./8D_rrs_Aug152024-Jan152025.npz')
rrs = data['rrs']
latitude = data['latitude']
longitude = data['longitude']
rrs.shape, latitude.shape, longitude.shape

In [ ]:
model = None
with open('svr_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
median = []
sd = []
predictions = []
dates = []

In [ ]:
# loop through all day-averaged images
index = 0
for d in date:
    # get single image
    img = rrs[index]
    img_shape = img.shape

    # reshape it to lat*long, bands
    img = img.reshape(-1, img_shape[2])

    # store nan locations
    valid_mask = ~np.isnan(img).any(axis=1)
    valid_data = img[valid_mask]

    # predict on non-nan pixels
    img_pred = model.predict(valid_data)

    # bring predictions to linear scale
    img_pred = np.expm1(img_pred)

    # merge predictions and nans, and reshape to 2d
    full_predictions = np.full(img_shape[0]*img_shape[1], np.nan)  # (lat*lon)
    full_predictions[valid_mask] = img_pred   
    img_pred = full_predictions.reshape(rrs[0].shape[0], rrs[0].shape[1])

    # save stats
    predictions.append(img_pred)
    median.append(np.nanmax(img_pred))
    sd.append(np.nanstd(img_pred))
    dates.append(np.datetime_as_string(d.values, 'D'))

    print(f"Inferences ran for {d.values} with prediction shape {img_pred.shape} and median cell count {median[index]}±{sd[index]}")
    index+=1
    
gc.collect()

## Plotting *K.Brevis* cell conc. maps

In [ ]:
plt.rcdefaults()
sns.set_theme(rc={"figure.dpi": 600})
sns.set_context("poster")
sns.set_style(style="ticks")

def set_font_size(fontsize):
    plt.rc('font', size=fontsize)
    plt.rc('axes', titlesize=fontsize)
    plt.rc('axes', labelsize=fontsize)
    plt.rc('xtick', labelsize=fontsize)
    plt.rc('ytick', labelsize=fontsize)
    plt.rc('legend', fontsize=fontsize)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import xarray as xr

set_font_size(20)
predictions_xr = []

rows, cols = 4, 5
fig = plt.figure(figsize=(25, 40))
gs = gridspec.GridSpec(rows, cols + 1, width_ratios=[1]*cols + [0.05], wspace=0.2, hspace=0.1)

for row in range(rows):
    for col in range(cols):
        index = row * cols + col
        ax = fig.add_subplot(gs[row, col], projection=ccrs.PlateCarree())

        # Prepare xarray object from predicted image
        img_pred_plot = xr.DataArray(
            predictions[index],  # shape (lat, lon)
            coords={"lat": latitude, "lon": longitude},
            dims=["lat", "lon"]
        )
        predictions_xr.append(img_pred_plot)

        # ax.coastlines()
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAND, edgecolor='black')
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAKES, alpha=0.5)
        ax.add_feature(cfeature.RIVERS)
        gl = ax.gridlines(draw_labels=True, alpha=0.3, linestyle="--")
        gl.top_labels = False
        gl.right_labels = False
        gl.left_labels = (col == 0)              # Only leftmost column
        gl.bottom_labels = (row == rows - 1)     # Only bottom row

        # Plot without colorbar for now
        plot = img_pred_plot.plot(
            x="lon", y="lat", cmap="jet", ax=ax,
            add_colorbar=False, vmin=0, vmax=120000
        )

        ax.set_title(str(dates[index]))

    # ---- Add ONE colorbar per row ----
    cax = fig.add_subplot(gs[row, -1])  # last column of each row
    cbar = fig.colorbar(plot, cax=cax)
    cbar.set_label("$K.\ Brevis$ cells/L", labelpad=20, fontsize=20)

plt.savefig('./weekly_kbrevis_spatial_variation.png')
plt.show()

In [ ]:
# predictions_xr = np.array(predictions_xr)
# predictions_xr[0]

## Prepare GIF from Maps

In [ ]:
set_font_size(16)

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
import matplotlib.ticker as mticker
import matplotlib.gridspec as gridspec
from PIL import Image
import io

lon2d, lat2d = np.meshgrid(longitude, latitude)

# Simulate a series of xarray.DataArray objects as prediction plots
img_pred_list = predictions_xr

# Generate frames for GIF
frames = []

for index, img_pred_plot in enumerate(img_pred_list):
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([longitude.min(), longitude.max(), latitude.min(), latitude.max()], crs=ccrs.PlateCarree())

    # Add map features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.RIVERS)

    # Gridlines and labels
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    # gl.xlabel_style = {'size': 10}
    # gl.ylabel_style = {'size': 10}
    gl.xlocator = mticker.FixedLocator(np.arange(-83, -79, 1))
    gl.ylocator = mticker.FixedLocator(np.arange(26, 30, 1))

    # Plot data
    plot = img_pred_plot.plot(
        x="lon", y="lat", cmap="jet", ax=ax, add_colorbar=True, cbar_kwargs={'label': "$K.\ Brevis$ cells/L"}, vmin=0, vmax=120000
    )

    ax.set_title(str(dates[index]))

    # Save figure to buffer
    buf = io.BytesIO()
    plt.savefig(buf, format="png", bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    frame = Image.open(buf)
    frames.append(frame)

# Save the frames as a GIF
gif_path = "./kbrevis_prediction_animation.gif"
frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=1000, loop=0)

gif_path

## *K.Brev* cell concentration Timeseries Plotting

In [ ]:
# ---- User Input: Define Transect ----
lat1 = 29.2
lon1 = -83.1
lat2 = 24.5
lon2 = -81.5

In [ ]:
set_font_size(10)
# ---- Generate Transect Points ----
npts = 100
lats = np.linspace(lat1, lat2, npts)
lons = np.linspace(lon1, lon2, npts)

# ---- Plot AVW Map and Transect ----
fig = plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-84.0, -80.0, 20.0, 30.0], crs=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.plot(lons, lats, color="black", linewidth=2, transform=ccrs.PlateCarree(), label="Transect")
ax.legend(loc="lower left")

# Plot AVW image
img = predictions_xr[0].plot(
    x="lon", y="lat",
    cmap="jet", vmin=450, vmax=490,
    ax=ax, add_colorbar=False
)

In [ ]:
lon2d, lat2d = np.meshgrid(longitude, latitude)

In [ ]:
lon2d, lat2d = np.meshgrid(longitude, latitude)
kb_cellcounts_8day = []
# ---- Plot spectra ----
for date in range(20):
    transect_kb = []
    for lat, lon in zip(lats, lons):
        # Get nearest index from 2D coordinate arrays
        dist = np.sqrt((lat2d - lat)**2 + (lon2d - lon)**2)
        i, j = np.unravel_index(np.argmin(dist), dist.shape)

        # Proceed with extraction
        transect_kb.append(predictions_xr[date].isel(lat=i, lon=j).item())
    kb_cellcounts_8day.append(transect_kb)

In [ ]:
len(predictions_xr)

In [ ]:
kb_cellcounts_8day = np.array(kb_cellcounts_8day)
kb_cellcounts_8day.shape

In [ ]:
kb_median = np.nanmedian(kb_cellcounts_8day, axis=1)
kb_std = np.nanstd(kb_cellcounts_8day, axis=1)
kb_median.shape, kb_std.shape

In [ ]:
plt.rcdefaults()
sns.set_theme(rc={"figure.dpi": 300})
sns.set_context("poster")
sns.set_style(style="ticks")

def set_font_size(fontsize):
    plt.rc('font', size=fontsize)
    plt.rc('axes', titlesize=fontsize)
    plt.rc('axes', labelsize=fontsize)
    plt.rc('xtick', labelsize=fontsize)
    plt.rc('ytick', labelsize=fontsize)
    plt.rc('legend', fontsize=fontsize)

In [ ]:
plt.rcParams.update({
    'lines.linewidth': 1.5,
    'axes.linewidth': 1.0,
    'xtick.major.width': 1.0,
    'ytick.major.width': 1.0,
    'xtick.minor.width': 0.8,
    'ytick.minor.width': 0.8,
    'xtick.direction': 'out',
    'ytick.direction': 'out',
})

In [ ]:
set_font_size(16)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Convert to matplotlib-compatible dates
dates_mpl = np.array(dates).astype('datetime64[D]').astype(datetime)

# Plotting
fig, ax = plt.subplots(figsize=(15, 7))

# Median as points instead of continuous line
ax.plot(dates_mpl, kb_median, 'o-', color='green', label='Median cells/L', linewidth=2)

# Shaded area for std (±1σ)
ax.fill_between(dates_mpl,
                kb_median - kb_std / 10,
                kb_median + kb_std / 10,
                color='green',
                alpha=0.2,
                label='±1 σ')

# Add vertical dotted line at Oct 2024
hurricane_date = datetime(2024, 10, 9)
ax.axvline(hurricane_date, color='red', linestyle='--', linewidth=1.5, label='Hurricane Milton')
ax.text(hurricane_date, ax.get_ylim()[1]*0.95, 'Hurricane Milton', color='red', rotation=90,
        verticalalignment='top', horizontalalignment='right')

# Formatting
ax.set_title("$K.Brev$ cell concentration for West FL Coast")
ax.set_ylabel("$K. Brev$ (cells/L)")
ax.grid(True, linestyle='--', alpha=0.5)

# Improve date formatting
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
fig.autofmt_xdate()

# Legend
ax.legend()

plt.tight_layout()
plt.show()

## Prepare GIF for *K.Brev* cell concentration timeseries

In [ ]:
set_font_size(24)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import imageio
from datetime import datetime
import io
from PIL import Image

# Simulate data
np.random.seed(42)
n_points = 20

# Define hurricane date (optional, only show after it occurs)
hurricane_date = datetime(2024, 10, 9).date()

# Create frames
frames = []
for i in range(1, n_points + 1):
    # Plotting
    fig, ax = plt.subplots(figsize=(15, 7))
    
    # Median as points instead of continuous line
    ax.plot(dates_mpl[:i], kb_median[:i], 'o-', color='green', label='Median cells/L', linewidth=2)
    
    # Shaded area for std (±1σ)
    ax.fill_between(dates_mpl[:i],
                    kb_median[:i] - kb_std[:i] / 10,
                    kb_median[:i] + kb_std[:i] / 10,
                    color='green',
                    alpha=0.2,
                    label='±1 σ')
    
    if dates_mpl[i-1] >= hurricane_date:
        ax.axvline(hurricane_date, color='red', linestyle='--', linewidth=1.5, label='Hurricane Milton')
        ax.text(hurricane_date, ax.get_ylim()[1]*0.95, 'Hurricane Milton', color='red', rotation=90,
        verticalalignment='top', horizontalalignment='right')

    # Formatting
    ax.set_title("$K.Brev$ cell concentration for West FL Coast")
    ax.set_ylabel("$K. Brev$ (cells/L)")
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Improve date formatting
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    fig.autofmt_xdate()
    
    # Legend
    ax.legend()

    plt.tight_layout()

    # Save figure to buffer
    buf = io.BytesIO()
    plt.savefig(buf, format="png", bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    frame = Image.open(buf)
    frames.append(frame)

# Save the frames as a GIF
gif_path = "./kbrevis_timeseries_animation_v2.gif"
frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=1000, loop=0)

gif_path

In [ ]:
# Load dataset
dataset = xr.open_mfdataset(fileset[0])

In [ ]:


rrs = dataset["Rrs"].sel({"wavelength": 709, "lon": slice(bbox[0],bbox[2]), "lat": slice(28.215572, 23.847017)})


In [ ]:
# ---- Make product plot ----
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=cartopy.crs.PlateCarree())
ax.gridlines(draw_labels={"bottom": "x", "left": "y"})
ax.coastlines()

# ---- Adjust colorbar limits ----
plot = rrs.plot(x="lon", y="lat", cmap="jet", vmin=0, vmax=0.002)

# ---- Crop the image to preferred location ----
plot.axes.set_xlim(-82.85257,-81.68223)
plot.axes.set_ylim(23.847017,28.215572)

# ---- Customize colorbar label ----
plot.colorbar.set_label("AVW (nm)", labelpad=10, fontsize=16, fontweight="bold")

In [ ]:
%%time

dataset["Rrs"].sel({"wavelength": 709, "lon": slice(-82.85257,-81.68223), "lat": slice(28.215572, 23.847017)}).load()

In [ ]:
b547 = Rrs.sel(wavelength=547, method='nearest')

Let's make a map of the AVW product to get a sense of what kind of water masses we're dealing with here, and explore what looks interesting. 

Pretty, right? This is a cool bloom that happens in the middle of a gyre, just north of Hawaii. Alright, let's go ahead and dive in and see what is going on beneath the AVW "hood".

Take a look at the map, and think of a starting and ending point. We're going to **build a transect** and extract the underlying data. I pre-filled some values, but feel free to put in your own start/end coordinates below, using the map grid as a reference. Keep in mind, we're in the Western Hemisphere here, so the longitudes are negative (-) values, e.g., 93°W = -93.0. Latitude is looking at the bright side of life and staying positive (for this scene anyway). 

In [ ]:
# ---- User Input: Define Transect ----
lat1 = 29.0
lon1 = -83.5
lat2 = 24.5
lon2 = -81.5

Let's go ahead and put that transect on the AVW map, 1) because we can, and 2) so we can verify that the transect is where we want it.

In [ ]:
# ---- Generate Transect Points ----
npts = 100
lats = np.linspace(lat1, lat2, npts)
lons = np.linspace(lon1, lon2, npts)

# ---- Plot AVW Map and Transect ----
fig = plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-84.0, -80.0, 20.0, 30.0], crs=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.plot(lons, lats, color="black", linewidth=2, transform=ccrs.PlateCarree(), label="Transect")
ax.legend(loc="lower left")

# Plot AVW image
img = avw[0].plot(
    x="lon", y="lat",
    cmap="jet", vmin=450, vmax=490,
    ax=ax, add_colorbar=False
)

Next, for every data point that was extracted along the transect, we're going to pull together 172 layers of Rrs information and see what each of those spectra look like. But wait, there's more! To help add some context to those spectra, we're going to make sure that we color code each spectra so that it corresponds to the colormap on our AVW image. In other words, a red-colored spectrum will represent data that you pulled from a red part of the map. 

This will generate two plots. The first plot will show the Rrs spectrum "as is" in units of inverse steradians (sr^-1). It is useful to examine this, because it tells you something about the *brightness* of the water. Sometimes, two spectral shapes may look very similar, but the relative magnitude of values can help you determine if you're looking at a more highly scattering water mass (higher relative magnitude) versus a highly absorbing water mass (lower relative magnitude). In the second plot, we divided the Rrs by the integrated area under the spectrum. This puts everything "equal" in terms of brightness, allowing you to focus more on absolute spectral shape differences. Both are informative in their own way.

Feel free to go back and build a new transect, explore, and get a feel for what different water masses look like.

In [ ]:
import pandas as pd

In [ ]:
rrs_pace_df = pd.DataFrame(columns=['Wavelength'])

In [ ]:
lon2d, lat2d = np.meshgrid(longitude, latitude)
lon2d

In [ ]:
# ---- Color setup ----
cmap = plt.get_cmap("jet")
norm = colors.Normalize(vmin=450, vmax=490)

# Prepare figure
fig, (ax_raw, ax_norm) = plt.subplots(1, 2, figsize=(14, 5), gridspec_kw={"wspace": 0.25})

# ---- For setting y-limits of normalized plot ----
ymin, ymax = np.inf, -np.inf

lon2d, lat2d = np.meshgrid(longitude, latitude)

# ---- Plot spectra ----
for lat, lon in zip(lats, lons):
    # Get nearest index from 2D coordinate arrays
    dist = np.sqrt((lat2d - lat)**2 + (lon2d - lon)**2)
    i, j = np.unravel_index(np.argmin(dist), dist.shape)

    # Proceed with extraction
    spectrum = Rrs.isel(lat=i, lon=j).values
    avw_val  = avw.isel(lat=i, lon=j).values

    # Normalize spectrum
    area = trapezoid(spectrum, wavelengths)
    spec_norm = spectrum / area if area > 0 else spectrum

    # Update y-limits for normalized spectra
    ymin = min(ymin, np.nanmin(spec_norm))
    ymax = max(ymax, np.nanmax(spec_norm))

    # Plot
    color = cmap(norm(avw_val))
    ax_raw.plot(wavelengths, spectrum, color=color, linewidth=0.8)
    ax_norm.plot(wavelengths, spec_norm, color=color, linewidth=0.8)

# ---- Shared colorbar ----
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=(ax_raw, ax_norm), location='right', pad=0.02)
cbar.set_label("AVW (nm)", fontsize=12)

# ---- Titles and labels ----
ax_raw.set_title("Raw Rrs Spectra Along Transect")
ax_norm.set_title("Area-Normalized Rrs Spectra")
ax_raw.set_xlabel("Wavelength (nm)")
ax_raw.set_ylabel("Rrs (1/sr)")
ax_norm.set_xlabel("Wavelength (nm)")
ax_norm.set_ylabel("Normalized Rrs")
ax_norm.set_ylim(ymin - 0.05 * (ymax - ymin), ymax + 0.05 * (ymax - ymin))

plt.show()

As you can likely see from your plots above, the optical properties of the open ocean can be variable under some conditions. So much so, in fact, that if we're looking to extract values from the satellite image for more quantitative analyses (e.g., matching up to a ship or animal observation), you're going to want a little context of the relative variability surrounding that pixel to account for spatial uncertainty. 

As our last exercise in this tutorial, we're going to define a single location in the image, and then extract a 5x5 box of pixels around that central location. With this information, we will plot the mean + standard deviation of the spectra in that 5x5 box. This is a common way scientists extract data for satellite calibration, validation, and algorithm development. Let's start by picking a point in the image.

In [ ]:
# ---- Extract mean Rrs spectrum over a 5x5 pixel box around a target point ----
print("Enter coordinates to extract Rrs over a 5x5 pixel box:")
target_lat = 27.0
target_lon = -158.0

Using that, we'll find the closest central point by subtracting your lat/lon input from all lat/lon values in the array. Take the absolute value of that, and the lowest number will yield your closest pixel matchup. We'll make a mini box that goes +2 pixels out in every direction, which translates to a 5x5 pixel cube that we can run our statistics on.  

In [ ]:
# Find nearest pixel
#lat_vals = rrs.latitude.values
#lon_vals = rrs.longitude.values
dist = np.abs(lat2d - target_lat) + np.abs(lon2d - target_lon)
i, j = np.unravel_index(dist.argmin(), lat2d.shape)

# Define 5x5 box bounds
half_box = 2  # for a 5x5 box
i_min = max(i - half_box, 0)
i_max = min(i + half_box + 1, lat2d.shape[0])
j_min = max(j - half_box, 0)
j_max = min(j + half_box + 1, lat2d.shape[1])

# Extract and compute statistics
rrs_box = Rrs.isel(lat=slice(i_min, i_max), lon=slice(j_min, j_max))
rrs_mean = rrs_box.mean(dim=("lat", "lon"))
rrs_std = rrs_box.std(dim=("lat", "lon"))
#wavelengths = rrs["wavelength_3d"]

# Plot mean spectrum with standard deviation
plt.figure(figsize=(10, 5))
plt.errorbar(wavelengths, rrs_mean, yerr=rrs_std, fmt='-o', capsize=3)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Remote Sensing Reflectance (Rrs)")
plt.title(f"Mean Rrs Spectrum ± 1σ at ({target_lat}, {target_lon})")
plt.grid(True)
plt.tight_layout()
plt.show()

Well folks, there you have it. Hyperspectral data, demystified. Well, maybe not entirely, but the tools here can be adapted to really help you explore the data and get a little more hands on with the raw reflectance information. Users often shy away from this information in favor of more geophysical variables that can be modeled and explained in the context of an ecosystem, like chlorophyll-a. This is completely understandable, but do keep in mind that every step you get away from the refelctance introduces additional uncertainties. Some folks have found that using raw reflectance values can outperform the use of more derived products, as in the case of [modeling Atlantic Sturgeon habitat](https://doi.org/10.1093/icesjms/fsx187). 